In [1]:
!pip install openai pinecone-client sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.0 MB/s eta 0:00:00


In [4]:
pip install --upgrade pinecone-client


In [5]:
import os
from pinecone import Pinecone, ServerlessSpec

# Replace these with your actual API key and environment
API_KEY = "YOUR_API_KEY"
ENVIRONMENT = "YOUR_PINECONE_ENV"

# Create a Pinecone instance
pc = Pinecone(
    api_key=API_KEY,
)

# Define the index name
index_name = "business-qa-bot"

# Check if the index exists, and create it if it doesn't
if index_name not in [idx.name for idx in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=768,  # Adjust dimension based on your embedding model
        metric="cosine",  # Use "cosine" or "euclidean" as needed
        spec=ServerlessSpec(
            cloud="aws",  # Cloud provider
            region=ENVIRONMENT,  # Region
        ),
    )

# Connect to the index
index = pc.Index(index_name)

# Verify the index connection
print(f"Connected to Pinecone index: {index_name}")


Connected to Pinecone index: business-qa-bot


In [6]:
from sentence_transformers import SentenceTransformer

# Initialize SentenceTransformer model for embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models as needed

# Example knowledge base (replace with your business data)
knowledge_base = [
    {"id": "1", "text": "Our company offers cloud-based solutions."},
    {"id": "2", "text": "We provide 24/7 customer support."},
    {"id": "3", "text": "We specialize in AI-powered analytics."},
    # Add more entries here
]

# Upsert data into the Pinecone index
for entry in knowledge_base:
    # Generate embeddings
    vector = embedding_model.encode(entry['text']).tolist()
    # Upsert data into Pinecone
    index.upsert([(entry['id'], vector, {"text": entry['text']})])

print("Data successfully upserted to Pinecone!")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 29 Dec 2024 06:41:47 GMT', 'Content-Type': 'application/json', 'Content-Length': '102', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '35', 'x-pinecone-request-id': '7298020394366118220', 'x-envoy-upstream-service-time': '35', 'x-envoy-immediate-health-check-fail': 'true', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 768","details":[]}


In [7]:
from pinecone import Pinecone, ServerlessSpec

# Delete the existing index if it exists
if index_name in [idx.name for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create a new index with the correct dimension
pc.create_index(
    name=index_name,
    dimension=384,  # Match the embedding model dimension
    metric="cosine",  # Similarity metric
    spec=ServerlessSpec(
        cloud="aws",
        region=ENVIRONMENT,
    ),
)

# Reconnect to the new index
index = pc.Index(index_name)
print(f"New Pinecone index '{index_name}' created and connected!")



New Pinecone index 'business-qa-bot' created and connected!


In [8]:
from sentence_transformers import SentenceTransformer

# Load the model (no authentication needed for public models)
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print("Model loaded successfully!")


Model loaded successfully!


In [9]:
#Resolve Hugging Face Warning

In [10]:
# Define your knowledge base
knowledge_base = [
    {"id": "1", "text": "Our company offers cloud-based solutions."},
    {"id": "2", "text": "We provide 24/7 customer support."},
    {"id": "3", "text": "We specialize in AI-powered analytics."},
]

# Embed and upsert data
for entry in knowledge_base:
    vector = embedding_model.encode(entry['text']).tolist()
    index.upsert([(entry['id'], vector, {"text": entry['text']})])

print("Data successfully upserted to Pinecone!")


Data successfully upserted to Pinecone!


In [12]:
def get_relevant_context(query, top_k=3):
    # Embed the query
    query_vector = embedding_model.encode(query).tolist()

    # Query Pinecone with correct keyword arguments
    results = index.query(
        vector=query_vector,
        top_k=top_k,
        include_metadata=True
    )

    # Extract and return relevant text
    return " ".join([match['metadata']['text'] for match in results['matches']])

# Test the updated function
query = "What services does your company provide?"
context = get_relevant_context(query)
print("Relevant context:", context)


Relevant context: Our company offers cloud-based solutions. We provide 24/7 customer support. We specialize in AI-powered analytics.


In [15]:
pip show openai


Name: openai
Version: 1.57.4
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [16]:
pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [17]:
import openai

# Set OpenAI API key
openai.api_key = "Your_openAI_API_KEY"


def generate_answer(query):
    # Retrieve relevant context from Pinecone
    context = get_relevant_context(query)

    # Use the new ChatCompletion API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use "gpt-4" if required
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides accurate answers to user queries based on context."},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
        ],
        max_tokens=200,
        temperature=0.7,
    )
    return response['choices'][0]['message']['content'].strip()

# Test the updated QA bot
query = "What services does your company offer?"
answer = generate_answer(query)
print("Answer:", answer)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
